# AMSC698K Homework 6
##### Elijah Kin & Noorain Noorani

In [1]:
import math
import numpy as np
import qiskit
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
qiskit.__version__

'1.2.4'

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer

### 1a - (60pts) Create a program for Shor's algorithm, i.e. for factorizing an integer N, including a quantum circuit to find the order r of a modulo $N (a^r := 1 mod N)$. 

First, create functions that perform the required processing steps (you may adapt the corresponding functions presented in class):

1) Create helper functions:

    (a) to check whether integer N is a perfect power ($N=p^q)$;

    (b) to check whether N and the base a (with $1<a<N$) are coprime;

    (c) to extract the exponent r and the factors of N.

In [3]:
def is_perfect_power(n):
    """
    Check if integer n is a perfect power (n = p^q with q > 1).
    
    Returns:
        (True, p, q) if n is a perfect power,
        (False, None, None) otherwise.
    """
    # Loop over possible exponents q starting at 2.
    # The upper limit for q is taken from log2(n) since 2 is the smallest possible base.
    max_exponent = int(math.log(n, 2)) + 1
    for q in range(2, max_exponent + 1):
        p = int(round(n ** (1.0 / q)))
        # Check if p^q exactly equals n
        if p ** q == n:
            return True, p, q
    return False, None, None

In [4]:
def are_coprime(n, a):
    """
    Check if integer n and base a (with 1 < a < n) are coprime.
    
    Returns:
        True if gcd(n, a) == 1, False otherwise.
    """
    if not (1 < a < n):
        raise ValueError("Base a must satisfy 1 < a < n")
    return math.gcd(n, a) == 1

In [5]:
def order_and_factors(n, a):
    """
    Compute the order r of a modulo n (i.e., the smallest positive integer r 
    such that a^r ≡ 1 mod n). Then, if r is even, attempt to extract non-trivial 
    factors of n by computing:
        factor1 = gcd(a^(r/2) - 1, n)
        factor2 = gcd(a^(r/2) + 1, n)
    
    Returns:
        r: the order of a modulo n,
        factors: a tuple (factor1, factor2) if r is even, otherwise None.
    """
    # Find the order r (smallest r such that a^r mod n == 1)
    r = None
    for i in range(1, n):
        if pow(a, i, n) == 1:
            r = i
            break

    if r is None:
        raise ValueError("No order found; check inputs for coprimality.")
    
    factors = None
    # Only proceed with factor extraction if r is even.
    if r % 2 == 0:
        # Compute a^(r/2) mod n
        x = pow(a, r // 2, n)
        # Calculate possible factors using the gcd method.
        factor1 = math.gcd(x - 1, n)
        factor2 = math.gcd(x + 1, n)
        factors = (factor1, factor2)
    
    return r, factors

In [6]:
N = 16  # Try a perfect power: 16 = 2^4
print("Checking if N =", N, "is a perfect power:")
is_pp, base, exponent = is_perfect_power(N)
if is_pp:
    print(f"  Yes, {N} = {base}^{exponent}")
else:
    print("  No, it is not a perfect power.")

# Example for coprimality check:
n = 15
a = 7
print(f"\nChecking if {n} and {a} are coprime:")
print("  Coprime?" , are_coprime(n, a))

# Example for order and factor extraction:
# Here we use n = 15 and a = 7. Note that 7 and 15 are coprime.
print(f"\nExtracting order and factors for n = {n} with base a = {a}:")
r, factors = order_and_factors(n, a)
print("  Order r =", r)
if factors:
    print("  Extracted factors:", factors)
else:
    print("  Order is not even; no factors extracted via this method.")

Checking if N = 16 is a perfect power:
  Yes, 16 = 4^2

Checking if 15 and 7 are coprime:
  Coprime? True

Extracting order and factors for n = 15 with base a = 7:
  Order r = 4
  Extracted factors: (3, 5)


In [ ]:
""" Function to check if N is of type q^p"""
def check_if_power(N):
    """ Check if N is a perfect power in O(n^3) time, n=ceil(logN) """
    b=2
    while (2**b) <= N:
        a = 1
        c = N
        while (c-a) >= 2:
            m = int( (a+c)/2 )

            if (m**b) < (N+1):
                p = int( (m**b) )
            else:
                p = int(N+1)

            if int(p) == int(N):
                print('N is {0}^{1}'.format(int(m),int(b)) )
                return True

            if p<N:
                a = int(m)
            else:
                c = int(m)
        b=b+1

    return False

""" Function to get the value a ( 1<a<N ), such that a and N are coprime. Starts by getting the smallest a possible
    This normally is be done fully randomly, we just did like this for user (professor) to have complete control 
    over the a value that gets selected """
def get_value_a(N):

    """ ok defines if user wants to used the suggested a (if ok!='0') or not (if ok=='0') """
    ok='0'

    """ Starting with a=2 """
    a=2

    """ Get the smallest a such that a and N are coprime"""
    while math.gcd(a,N)!=1:
        a=a+1

    """ Store it as the smallest a possible """
    smallest_a = a

    """ Ask user if the a found is ok, if not, then increment and find the next possibility """  
    ok = input('Is the number {0} ok for a? Press 0 if not, other number if yes: '.format(a))
    if ok=='0':
        if(N==3):
            print('Number {0} is the only one you can use. Using {1} as value for a\n'.format(a,a))
            return a
        a=a+1

    """ Cycle to find all possibilities for a not counting the smallest one, until user says one of them is ok """
    while ok=='0':
        
        """ Get a coprime with N """
        while math.gcd(a,N)!=1:
            a=a+1
    
        """ Ask user if ok """
        ok = input('Is the number {0} ok for a? Press 0 if not, other number if yes: '.format(a))

        """ If user says it is ok, then exit cycle, a has been found """
        if ok!='0':
            break
        
        """ If user says it is not ok, increment a and check if are all possibilites checked.  """
        a=a+1

        """ If all possibilities for a are rejected, put a as the smallest possible value and exit cycle """
        if a>(N-1):
            print('You rejected all options for value a, selecting the smallest one\n')
            a=smallest_a
            break

    """ Print the value that is used as a """
    print('Using {0} as value for a\n'.format(a))

    return a

""" Function to apply the continued fractions to find r and the gcd to find the desired factors"""
def get_factors(x_value,t_upper,N,a):

    if x_value<=0:
        print('x_value is <= 0, there are no continued fractions\n')
        return False

    print('Running continued fractions for this case\n')

    """ Calculate T and x/T """
    T = pow(2,t_upper)

    x_over_T = x_value/T

    """ Cycle in which each iteration corresponds to putting one more term in the
    calculation of the Continued Fraction (CF) of x/T """

    """ Initialize the first values according to CF rule """
    i=0
    b = np.array('i')
    t = np.array('f')

    b.append(math.floor(x_over_T))
    t.append(x_over_T - b[i])

    while i>=0:

        """From the 2nd iteration onwards, calculate the new terms of the CF based
        on the previous terms as the rule suggests"""

        if i>0:
            b.append( math.floor( 1 / (t[i-1]) ) ) 
            t.append( ( 1 / (t[i-1]) ) - b[i] )

        """ Calculate the CF using the known terms """

        aux = 0
        j=i
        while j>0:    
            aux = 1 / ( b[j] + aux )      
            j = j-1
        
        aux = aux + b[0]

        """Get the denominator from the value obtained"""
        frac = fractions.Fraction(aux).limit_denominator()
        den=frac.denominator

        print('Approximation number {0} of continued fractions:'.format(i+1))
        print("Numerator:{0} \t\t Denominator: {1}\n".format(frac.numerator,frac.denominator))

        """ Increment i for next iteration """
        i=i+1

        if (den%2) == 1:
            if i>=15:
                print('Returning because have already done too much tries')
                return False
            print('Odd denominator, will try next iteration of continued fractions\n')
            continue
    
        """ If denominator even, try to get factors of N """

        """ Get the exponential a^(r/2) """

        exponential = 0

        if den<1000:
            exponential=pow(a , (den/2))
        
        """ Check if the value is too big or not """
        if math.isinf(exponential)==1 or exponential>1000000000:
            print('Denominator of continued fraction is too big!\n')
            aux_out = input('Input number 1 if you want to continue searching, other if you do not: ')
            if aux_out != '1':
                return False
            else:
                continue

        """If the value is not to big (infinity), then get the right values and
        do the proper gcd()"""

        putting_plus = int(exponential + 1)

        putting_minus = int(exponential - 1)
    
        one_factor = math.gcd(putting_plus,N)
        other_factor = math.gcd(putting_minus,N)
    
        """ Check if the factors found are trivial factors or are the desired
        factors """

        if one_factor==1 or one_factor==N or other_factor==1 or other_factor==N:
            print('Found just trivial factors, not good enough\n')
            """ Check if the number has already been found, use i-1 because i was already incremented """
            if t[i-1]==0:
                print('The continued fractions found exactly x_final/(2^(2n)) , leaving funtion\n')
                return False
            if i<15:
                aux_out = input('Input number 1 if you want to continue searching, other if you do not: ')
                if aux_out != '1':
                    return False       
            else:
                """ Return if already too much tries and numbers are huge """ 
                print('Returning because have already done too many tries\n')
                return False         
        else:
            print('The factors of {0} are {1} and {2}\n'.format(N,one_factor,other_factor))
            print('Found the desired factors!\n')
            return True

2) Create quantum circuits:

- to perform QFT and inverse QFT on N qubits;
- to perform the double-controlled modular addition of $a$ in the Fourier space and its inverse;
- to perform controlled modular multiplication by $a$ 

In [8]:
""" Function to create QFT """
def create_QFT(circuit,up_reg,n,with_swaps):
    i=n-1
    """ Apply the H gates and Cphases"""
    """ The Cphases with |angle| < threshold are not created because they do 
    nothing. The threshold is put as being 0 so all CPhases are created,
    but the clause is there so if wanted just need to change the 0 of the
    if-clause to the desired value """
    while i>=0:
        circuit.h(up_reg[i])        
        j=i-1  
        while j>=0:
            if (np.pi)/(pow(2,(i-j))) > 0:
                circuit.cp( (np.pi)/(pow(2,(i-j))) , up_reg[i] , up_reg[j] )
                j=j-1   
        i=i-1  

    """ If specified, apply the Swaps at the end """
    if with_swaps==1:
        i=0
        while i < ((n-1)/2):
            circuit.swap(up_reg[i], up_reg[n-1-i])
            i=i+1

In [9]:
def create_inverse_QFT(circuit, up_reg, n, with_swaps = True):
    if with_swaps:
        i=0
        while i < ((n-1)/2):
            circuit.swap(up_reg[i], up_reg[n-1-i])
            i=i+1
    i=0
    while i<n:
        circuit.h(up_reg[i])
        if i != n-1:
            j=i+1
            y=i
            while y>=0:
                 if (np.pi)/(pow(2,(j-y))) > 0:
                    circuit.cp( - (np.pi)/(pow(2,(j-y))) , up_reg[j] , up_reg[y] )
                    y=y-1   
        i=i+1

In [10]:
"""Function that calculates the array of angles to be used in the addition in Fourier Space"""
def getAngles(a,N):
    s=bin(int(a))[2:].zfill(N) 
    angles=np.zeros([N])
    for i in range(0, N):
        for j in range(i,N):
            if s[j]=='1':
                angles[N-i-1]+=math.pow(2, -(j-i))
        angles[N-i-1]*=np.pi
    return angles

"""Creation of a doubly controlled phase gate"""
def ccphase(circuit,angle,ctl1,ctl2,tgt):
    circuit.cp(angle/2,ctl1,tgt)
    circuit.cx(ctl2,ctl1)
    circuit.cp(-angle/2,ctl1,tgt)
    circuit.cx(ctl2,ctl1)
    circuit.cp(angle/2,ctl2,tgt)

In [11]:
"""Creation of the circuit that performs addition by a in Fourier Space"""
"""Can also be used for subtraction by setting the parameter inv to a value different from 0"""
def phiADD(circuit,q,a,N,inv):
    angle=getAngles(a,N)
    for i in range(0,N):
        if inv==0:
            circuit.p(angle[i],q[i])
        else:
            circuit.p(-angle[i],q[i])

"""Single controlled version of the phiADD circuit"""
def cphiADD(circuit,q,ctl,a,n,inv):
    angle=getAngles(a,n)
    for i in range(0,n):
        if inv==0:
            circuit.cp(angle[i],ctl,q[i])
        else:
            circuit.cp(-angle[i],ctl,q[i])

"""Doubly controlled version of the phiADD circuit"""      
def ccphiADD(circuit,q,ctl1,ctl2,a,n,inv):
    angle=getAngles(a,n)
    for i in range(0,n):
        if inv==0:
            ccphase(circuit,angle[i],ctl1,ctl2,q[i])
        else:
            ccphase(circuit,-angle[i],ctl1,ctl2,q[i])
        
"""Circuit that implements doubly controlled modular addition by a"""
def ccphiADDmodN(circuit, q, ctl1, ctl2, aux, a, N, n):
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)
    phiADD(circuit, q, N, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.cx(q[n-1],aux)
    create_QFT(circuit,q,n,0)
    cphiADD(circuit, q, aux, N, n, 0)
    
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.x(q[n-1])
    circuit.cx(q[n-1], aux)
    circuit.x(q[n-1])
    create_QFT(circuit,q,n,0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)

"""Circuit that implements the inverse of doubly controlled modular addition by a"""
def ccphiADDmodN_inv(circuit, q, ctl1, ctl2, aux, a, N, n):
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.x(q[n-1])
    circuit.cx(q[n-1],aux)
    circuit.x(q[n-1])
    create_QFT(circuit, q, n, 0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)
    cphiADD(circuit, q, aux, N, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.cx(q[n-1], aux)
    create_QFT(circuit, q, n, 0)
    phiADD(circuit, q, N, n, 0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)

In [12]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

"""Circuit that implements single controlled modular multiplication by a"""
def cMULTmodN(circuit, ctl, q, aux, a, N, n):
    create_QFT(circuit,aux,n+1,0)
    for i in range(0, n):
        ccphiADDmodN(circuit, aux, q[i], ctl, aux[n+1], (2**i)*a % N, N, n+1)
    create_inverse_QFT(circuit, aux, n+1, 0)

    for i in range(0, n):
        circuit.cswap(ctl,q[i],aux[i])

    a_inv = modinv(a, N)
    create_QFT(circuit, aux, n+1, 0)
    i = n-1
    while i >= 0:
        ccphiADDmodN_inv(circuit, aux, q[i], ctl, aux[n+1], math.pow(2,i)*a_inv % N, N, n+1)
        i -= 1
    create_inverse_QFT(circuit, aux, n+1, 0)

3) Create a program that performs the factorization:

- get a positive odd integer N (user input) and check whether N is a perfect power (function 1a);
- get base a and check whether a and N are coprime (function 1b);
with n as the number of bits in N, create 3 quantum registers: a n qubit register (initialized to 1) and a n+2 auxiliary qubit register for performing modular multiplication, and a 2n qubit register to perform the inverse QFT (initialized to uniform superposition);
- apply the modular multiplication gate of power 2k controlled by qubit k (for each k=0,2n) in the 2n qubit register;
- apply the inverse QFT on the 2n qubit register and read it out (measure all 2n qubits);
- extract the exponent r and the factors of N from the measured data (function 1c)

In [13]:
 print('Forcing N=15 and a=4 because its the fastest case, please read top of source file for more info')
N=15
a=4

""" Get n value used in Shor's algorithm, to know how many qubits are used """
n = math.ceil(math.log(N,2))

print('Total number of qubits used: {0}\n'.format(4*n+2))

""" Create quantum and classical registers """

"""auxilliary quantum register used in addition and multiplication"""
aux = QuantumRegister(n+2)
"""quantum register where the sequential QFT is performed"""
up_reg = QuantumRegister(2*n)
"""quantum register where the multiplications are made"""
down_reg = QuantumRegister(n)
"""classical register where the measured values of the QFT are stored"""
up_classic = ClassicalRegister(2*n)

""" Create Quantum Circuit """
circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic)

""" Initialize down register to 1 and create maximal superposition in top register """
circuit.h(up_reg)
circuit.x(down_reg[0])

""" Apply the multiplication gates as showed in the report in order to create the exponentiation """
for i in range(0, 2*n):
    cMULTmodN(circuit, up_reg[i], down_reg, aux, int(pow(a, pow(2, i))), N, n)

""" Apply inverse QFT """
create_inverse_QFT(circuit, up_reg, 2*n ,1)

""" Measure the top qubits, to get x value"""
circuit.measure(up_reg,up_classic)

number_shots = 1000

print('Executing the circuit {0} times for N={1} and a={2}\n'.format(number_shots,N,a))

backend = Aer.get_backend('aer_simulator')
job = backend.run(circuit, shots=number_shots)
sim_result = job.result()
counts_result = sim_result.get_counts()

i=0
while i < len(counts_result):
    print('Result \"{0}\" happened {1} times out of {2}'.format(list(sim_result.get_counts().keys())[i],list(sim_result.get_counts().values())[i],number_shots))
    i=i+1

""" An empty print just to have a good display in terminal """
print(' ')

""" Initialize this variable """
prob_success=0

""" For each simulation result, print proper info to user and try to calculate the factors of N"""
i=0
while i < len(counts_result):

    """ Get the x_value from the final state qubits """
    output_desired = list(sim_result.get_counts().keys())[i]
    x_value = int(output_desired, 2)
    prob_this_result = 100 * ( int( list(sim_result.get_counts().values())[i] ) ) / (number_shots)

    print("------> Analysing result {0}. This result happened in {1:.4f} % of all cases\n".format(output_desired,prob_this_result))

    """ Print the final x_value to user """
    print('In decimal, x_final value for this result is: {0}\n'.format(x_value))

    """ Get the factors using the x value obtained """   
    success=get_factors(int(x_value),int(2*n),int(N),int(a))
    
    if success==True:
        prob_success = prob_success + prob_this_result

    i=i+1

print("\nUsing a={0}, found the factors of N={1} in {2:.4f} % of the cases\n".format(a,N,prob_success))


Forcing N=15 and a=4 because its the fastest case, please read top of source file for more info
Total number of qubits used: 18

Executing the circuit 1000 times for N=15 and a=4

Result "10000000" happened 475 times out of 1000
Result "00000000" happened 525 times out of 1000
 
------> Analysing result 10000000. This result happened in 47.5000 % of all cases

In decimal, x_final value for this result is: 128

Running continued fractions for this case



NameError: name 'array' is not defined

### 1b - (40pts) Test your program on N=15,35,55, then submit the circuit first to "ionq_simulator" without and with noise model, check the results and then run it on an IonQ QPU.

### 1c - (extra credit 30pts) Finally, choose a number 156 < N < 256, which is the product of 2 primes, and factorize N using the IonQ Forte-1 QPU.